# Imports

In [47]:
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image


# Constants and hyperparameters

In [48]:
batch_size = 64
num_channels = 1
num_classes = 4
image_size = 600
latent_dim = 128

# Load data

In [49]:
"""(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
all_digits = np.concatenate([x_train, x_test])
all_labels = np.concatenate([y_train, y_test])
print("Training X MNIST: " + str(x_train.shape))
print("Training Y MNIST: " + str(y_train.shape))
print("Test X MNIST: " + str(x_test.shape))
print("Test Y MNIST: " + str(y_test.shape))
print("All X MNIST: " + str(all_digits.shape))
print("All Y MNIST: " + str(all_labels.shape))"""

'(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()\nall_digits = np.concatenate([x_train, x_test])\nall_labels = np.concatenate([y_train, y_test])\nprint("Training X MNIST: " + str(x_train.shape))\nprint("Training Y MNIST: " + str(y_train.shape))\nprint("Test X MNIST: " + str(x_test.shape))\nprint("Test Y MNIST: " + str(y_test.shape))\nprint("All X MNIST: " + str(all_digits.shape))\nprint("All Y MNIST: " + str(all_labels.shape))'

In [50]:
import os
os.chdir("/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/FagProject")

In [51]:
submissions = pd.read_csv('submissions.csv')
data = submissions[0:1000]
upvotes = data["Score"].to_numpy()
logupvotes = np.log(upvotes+1)
data["Log_Upvotes"]=logupvotes

quantiles=[0.25,0.5,0.75]
labels=np.quantile(data["Log_Upvotes"],quantiles)

/var/folders/_6/l3776shn2kq8j4f7hm3qr5bh0000gn/T/ipykernel_15269/294398999.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["Log_Upvotes"]=logupvotes


In [52]:
os.chdir("/Users/clarasofiechristiansen/Documents/Clara/DTU/Data_Fagprojekt/FagProject/resized_images/")
X = np.zeros((len(data),600,600))
y=np.zeros(len(data))
for i, row in data.iterrows():
    temp_labels=np.append(labels,row["Log_Upvotes"])
    y[i]=int(np.where(np.sort(temp_labels)==row["Log_Upvotes"])[0][0])
    image = np.asarray(Image.open('EarthPorn-' + str(row["ID"]) + '.png').convert('L'))
    X[i] = image
print(np.count_nonzero(y==0))
print(np.count_nonzero(y==1))
print(np.count_nonzero(y==2))
print(np.count_nonzero(y==3))

275
230
245
250


In [53]:
X = X.astype("float32") / 255.0
X = np.reshape(X, (-1, 600, 600, 1))
y = keras.utils.to_categorical(y, 4)


dataset = tf.data.Dataset.from_tensor_slices((X, y))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

print(f"Shape of training images: {X.shape}")
print(f"Shape of training labels: {y.shape}")




Shape of training images: (1000, 600, 600, 1)
Shape of training labels: (1000, 4)


In [54]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

132 5


In [58]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((image_size, image_size, discriminator_in_channels)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)
"""Version 2
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (5, 5), strides=(10, 10), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (5, 5), padding="same", activation="sigmoid"),
        layers.UpSampling2D(size=(6, 6))
    ],
    name="generator",
)
"""
"""Version 1
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (5, 5), strides=(10, 10), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (5, 5), padding="same", activation="sigmoid"),
        layers.UpSampling2D(size=(12, 12))
    ],
    name="generator",
)"""
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (5, 5), strides=(15, 15), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (5, 5), padding="same", activation="sigmoid"),
        layers.Cropping2D(cropping=((90, 90), (90, 90))),
        layers.UpSampling2D(size=(2, 2))
    ],
    name="generator",
)

In [61]:
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        layers.Dense(7 * 7 * generator_in_channels),
        layers.LeakyReLU(alpha=0.2),
        layers.Reshape((7, 7, generator_in_channels)),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(1, (7, 7), padding="same", activation="sigmoid"),
        layers.Cropping2D(cropping=((1, 1), (1, 1))),
        layers.UpSampling2D(size=(85, 85))
    ],
    name="generator",
)

discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((600, 600, 1)),
        layers.Conv2D(64, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, (3, 3), strides=(2, 2), padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.GlobalMaxPooling2D(),
        layers.Dense(1),
    ],
    name="discriminator",
)


In [62]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [63]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.legacy.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

cond_gan.fit(dataset, epochs=20)

Epoch 1/20


ValueError: in user code:

    File "/Users/clarasofiechristiansen/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/clarasofiechristiansen/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/clarasofiechristiansen/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/var/folders/_6/l3776shn2kq8j4f7hm3qr5bh0000gn/T/ipykernel_15269/4054037992.py", line 47, in train_step
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)

    ValueError: Dimension 1 in both shapes must be equal, but are 2210 and 600. Shapes are [?,2210,2210] and [?,600,600]. for '{{node concat_1}} = Conca